In [1]:
import os
import re
import time
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

from itertools import permutations
import requests
import json

pd.set_option("display.max_rows", None, "display.max_columns", None)

C:\Users\mtayl\AppData\Local\Temp\ipykernel_12276\1985660408.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
mens_path = os.path.abspath('../../data/teams.csv')
mens_df = pd.read_csv(mens_path)
mens_df.head()

,School,"City, State",SR key,NCAA key,NCAA School,NCAA Name,background-color
0,Abilene Christian,"Abilene, Texas",abilene-christian,abilene-christian,Abilene Christian,Abilene Christian University,#582C83
1,Air Force,"USAF Academy, Colorado",air-force,air-force,Air Force,Air Force Academy,#0032A0
2,Akron,"Akron, Ohio",akron,akron,Akron,University of Akron,#0F192B
3,Alabama,"Tuscaloosa, Alabama",alabama,alabama,Alabama,University of Alabama,#9D2235
4,Alabama A&M,"Normal, Alabama",alabama-am,alabama-am,Alabama A&M,Alabama A&M University,#862633


In [3]:
womens_path = os.path.abspath('../../data/womens_teams.csv')
womens_df = pd.read_csv(womens_path)
womens_df.head()

,School,"City, State",SR key,NCAA key,NCAA School,NCAA Name,background-color
0,Abilene Christian,"Abilene, Texas",abilene-christian,abilene-christian,Abilene Christian,Abilene Christian University,#582C83
1,Air Force,"USAF Academy, Colorado",air-force,air-force,Air Force,Air Force Academy,#0032A0
2,Akron,"Akron, Ohio",akron,akron,Akron,University of Akron,#0F192B
3,Alabama,"Tuscaloosa, Alabama",alabama,alabama,Alabama,University of Alabama,#9D2235
4,Alabama A&M,"Normal, Alabama",alabama-am,alabama-am,Alabama A&M,Alabama A&M University,#862633


In [4]:
MENS_SR_SCHOOL_KEYS = [row.to_dict().get('SR key') for index, row in mens_df.iterrows()]
WOMENS_SR_SCHOOL_KEYS = [row.to_dict().get('SR key') for index, row in womens_df.iterrows()]

def get_gamelog_basic_url(school_key, season, isWomens = False):
    type = 'women' if isWomens else 'men'
    return f'https://www.sports-reference.com/cbb/schools/{school_key}/{type}/{season}-gamelogs.html'

def get_gamelog_advanced_url(school_key, season, isWomens = False):
    type = 'women' if isWomens else 'men'
    return f'https://www.sports-reference.com/cbb/schools/{school_key}/{type}/{season}-gamelogs-advanced.html'

def get_team_season_file_path(school_key, season, filename, isWomens = False):
    type = 'women' if isWomens else 'men'
    file_path = os.path.abspath(f'../../data/seasons/{season}/{school_key}/{type}/{filename}')

    dir = os.path.dirname(file_path)
    if not os.path.exists(dir):
        os.makedirs(dir)
    
    return file_path

## Download gamelogs HTML
Download the basic and advanced gamelog html for each team

In [5]:
def download_gamelog(school_key, season, isWomens = False):
    basic_url, advanced_url = get_gamelog_basic_url(school_key, season, isWomens), get_gamelog_advanced_url(school_key, season, isWomens)

    time.sleep(3) # Delay for 3 seconds
    basic_html = requests.get(basic_url).content
    time.sleep(3) # Delay for 3 seconds
    advanced_html = requests.get(advanced_url).content

    basic_file_path = get_team_season_file_path(school_key, season, f'{school_key}_basic.html', isWomens)
    advanced_file_path = get_team_season_file_path(school_key, season, f'{school_key}_advanced.html', isWomens)

    with open(basic_file_path, 'w') as file:
        file.write(basic_html.decode('utf-8'))
    with open(advanced_file_path, 'w') as file:
        file.write(advanced_html.decode('utf-8'))

def download_gamelogs_for_single_season(season, isWomens = False):
    teams = WOMENS_SR_SCHOOL_KEYS if isWomens else MENS_SR_SCHOOL_KEYS
    for school_key in tqdm(teams, unit=f'school ({season})'):
        download_gamelog(school_key, season, isWomens)

def download_gamelogs(seasons, isWomens = False):
    for season in seasons:
        download_gamelogs_for_single_season(season, isWomens)

## Change Opp name column to Opp key
The Opp name does not always match the school name we have saved in the team data, thus we will reparse the tables to instead use the SR keys

In [6]:
def get_opposing_school_keys(school_key, season, advanced = False, isWomens = False):
    opp_school_keys = []
    type = 'advanced' if advanced else 'basic'
    html_file_path = get_team_season_file_path(school_key, season, f'{school_key}_{type}.html', isWomens)

    with open(html_file_path, 'r') as file:
        soup = BeautifulSoup(file, 'html.parser')
        table = soup.find("table")
        rows = table.find_all('tr')
    
        for row in rows[2:]:
            try:
                link = row.find_all('td')[2].find('a')['href']
                key = re.search(r'/schools/([^/]+)/', link).group(1)

                if isWomens and '_w' in key:
                    key = key.replace('_w', '')
                opp_school_keys.append(key)
            except IndexError:
                # repeating header row
                continue
            except TypeError:
                #  opponent is missing
                opp_school_keys.append('')
    
    return opp_school_keys

## Basic gamelog CSV
Extract the basic gamelog to csv

In [7]:
def create_basic_gamelog(school_key, season, isWomens = False):
    file_path = get_team_season_file_path(school_key, season, f'{school_key}_basic.html', isWomens)

    team_df = pd.read_html(file_path)[0]

    # drop columns from 'Defensive Four Factors'
    opponent_columns = [column for column in team_df.columns if 'Opponent' in column[0]]
    team_df = team_df.drop(opponent_columns, axis=1)

    # Use second level column names
    team_df.columns = [column[1] for column in team_df.columns]

    # rename to location column
    team_df = team_df.rename(columns={'Unnamed: 2_level_1': 'Location'})

    # remove unneeded columns
    unneeded_columns = [column for column in team_df.columns if 'Unnamed' in column] + ['G']
    team_df = team_df.drop(unneeded_columns, axis=1)

    # Drop repeating header rows
    team_df = team_df[team_df.Tm != 'Tm']
    team_df = team_df[team_df.FG != 'School']

    # rename repeating 'Opp' column
    index = team_df.columns.to_list().index('Opp')
    team_df.columns.values[index] = 'Opp name'

    # Opp names to Opp keys
    opp_school_keys = get_opposing_school_keys(school_key, season, False, isWomens)
    # Shape for both must match same rows
    assert team_df.shape[0] == len(opp_school_keys)
    team_df['Opp name'] = opp_school_keys
    team_df = team_df.rename(columns={'Opp name': 'Opp key'})

    # save file
    csv_file_path = get_team_season_file_path(school_key, season, f'{school_key}_basic.csv', isWomens)
    team_df.to_csv(csv_file_path, index=False)

def create_basic_gamelogs_for_single_season(season, isWomens = False):
    teams = WOMENS_SR_SCHOOL_KEYS if isWomens else MENS_SR_SCHOOL_KEYS
    for school_key in tqdm(teams, unit=f'school ({season} basic csv)'):
        try:
            create_basic_gamelog(school_key, season, isWomens)
        except ValueError:
            continue

def create_basic_gamelogs(seasons, isWomens = False):
    for season in seasons:
        create_basic_gamelogs_for_single_season(season, isWomens)

In [8]:
# create_basic_gamelogs(['2020', '2021', '2022', '2023'], isWomens=True)

Quick visual verification:

In [9]:
school_key, season = 'connecticut', 2020
file_path = get_team_season_file_path(school_key, season, f'{school_key}_basic.csv', isWomens=True)
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,2019-11-10,NaN,california,W,72,61,27,65,0.415,5,15,0.333,13,21,0.619,9,38,12,10,6,10,12
1,2019-11-13,@,vanderbilt,W,64,51,27,65,0.415,6,23,0.261,4,7,0.571,11,32,17,16,5,15,9
2,2019-11-17,@,temple,W,83,54,31,63,0.492,8,22,0.364,13,16,0.813,11,34,21,7,6,11,8
3,2019-11-19,NaN,virginia,W,83,44,30,59,0.508,9,23,0.391,14,22,0.636,9,37,21,11,7,10,13
4,2019-11-24,@,ohio-state,W,73,62,29,61,0.475,8,21,0.381,7,8,0.875,8,39,19,7,4,14,9


## Advanced gamelog CSV
Extract the advanced gamelog to csv

In [10]:
def create_advanced_gamelog(school_key, season, isWomens = False):
    file_path = get_team_season_file_path(school_key, season, f'{school_key}_advanced.html', isWomens)

    team_df = pd.read_html(file_path)[0]

    # drop columns from 'Defensive Four Factors'
    defensive_columns = [column for column in team_df.columns if 'Defensive' in column[0]]
    team_df = team_df.drop(defensive_columns, axis=1)

    # Use second level column names
    team_df.columns = [column[1] for column in team_df.columns]

    # rename to location column
    team_df = team_df.rename(columns={'Unnamed: 2_level_1': 'Location'})

    # remove unneeded columns
    unneeded_columns = [column for column in team_df.columns if 'Unnamed' in column] + ['G']
    team_df = team_df.drop(unneeded_columns, axis=1)

    # Drop repeating header rows
    team_df = team_df[team_df.Tm != 'Tm']
    team_df = team_df[team_df['eFG%'] != 'Offensive Four Factors']

    # rename repeating 'Opp' column
    index = team_df.columns.to_list().index('Opp')
    team_df.columns.values[index] = 'Opp name'

    # Opp names to Opp keys
    opp_school_keys = get_opposing_school_keys(school_key, season, True, isWomens)
    # Shape for both must match same rows
    assert team_df.shape[0] == len(opp_school_keys)
    team_df['Opp name'] = opp_school_keys
    team_df = team_df.rename(columns={'Opp name': 'Opp key'})

    # save file
    csv_file_path = get_team_season_file_path(school_key, season, f'{school_key}_advanced.csv', isWomens)
    team_df.to_csv(csv_file_path, index=False)

def create_advanced_gamelogs_for_single_season(season, isWomens = False):
    teams = WOMENS_SR_SCHOOL_KEYS if isWomens else MENS_SR_SCHOOL_KEYS
    for school_key in tqdm(teams, unit=f'school ({season} advanced csv)'):
        try:
            create_advanced_gamelog(school_key, season, isWomens)
        except ValueError:
            continue

def create_advanced_gamelogs(seasons, isWomens = False):
    for season in seasons:
        create_advanced_gamelogs_for_single_season(season, isWomens)

In [11]:
# create_advanced_gamelogs(['2020', '2021', '2022', '2023'], isWomens=True)

Quick visual verification:

In [12]:
school_key, season = 'connecticut', 2020
file_path = get_team_season_file_path(school_key, season, f'{school_key}_advanced.csv', isWomens=True)
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,2019-11-10,NaN,california,W,72,61,96.7,81.9,74.4,0.323,0.231,0.480,50.7,44.4,13.4,13.3,0.454,11.8,25.7,0.200
1,2019-11-13,@,vanderbilt,W,64,51,87.3,69.6,73.0,0.108,0.354,0.468,48.5,63.0,21.8,10.4,0.462,18.0,31.4,0.062
2,2019-11-17,@,temple,W,83,54,119.9,78.0,69.2,0.254,0.349,0.588,56.7,67.7,10.1,17.1,0.556,13.5,36.7,0.206
3,2019-11-19,NaN,virginia,W,83,44,117.6,62.4,70.6,0.373,0.390,0.598,54.4,70.0,15.6,17.1,0.585,12.6,31.0,0.237
4,2019-11-24,@,ohio-state,W,73,62,100.0,84.9,73.0,0.131,0.344,0.563,54.2,65.5,9.6,10.5,0.541,17.8,26.7,0.115


## Combine basic and advanced gamelog CSVs

In [13]:
def combine_basic_advanced_gamelog(school_key, season, isWomens = False):
    basic_file_path = get_team_season_file_path(school_key, season, f'{school_key}_basic.csv', isWomens)
    advanced_file_path = get_team_season_file_path(school_key, season, f'{school_key}_advanced.csv', isWomens)

    basic_team_df, advanced_team_df = pd.read_csv(basic_file_path), pd.read_csv(advanced_file_path)

    merged_team_df = pd.merge(basic_team_df, advanced_team_df, on=['Date', 'Location', 'Opp key', 'W/L', 'Tm', 'Opp'])

    # fill NaN location values to 'H' to represent Home
    merged_team_df['Location'] = merged_team_df['Location'].fillna('H')
    
    csv_file_path = get_team_season_file_path(school_key, season, f'{school_key}_merged.csv', isWomens)
    merged_team_df.to_csv(csv_file_path, index=False)

def combine_basic_advanced_gamelogs_for_single_season(season, isWomens = False):
    teams = WOMENS_SR_SCHOOL_KEYS if isWomens else MENS_SR_SCHOOL_KEYS
    for school_key in tqdm(teams, unit=f'school ({season} merged csv)'):
        try:
            combine_basic_advanced_gamelog(school_key, season, isWomens)
        except FileNotFoundError:
            continue

def combine_basic_advanced_gamelogs(seasons, isWomens = False):
    for season in seasons:
        combine_basic_advanced_gamelogs_for_single_season(season, isWomens)

In [14]:
# combine_basic_advanced_gamelogs(['2020', '2021', '2022', '2023'], isWomens=True)

Quick visual verification:

In [15]:
school_key, season = 'connecticut', 2020
file_path = get_team_season_file_path(school_key, season, f'{school_key}_merged.csv', isWomens=True)
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,2019-11-10,H,california,W,72,61,27,65,0.415,5,15,0.333,13,21,0.619,9,38,12,10,6,10,12,96.7,81.9,74.4,0.323,0.231,0.480,50.7,44.4,13.4,13.3,0.454,11.8,25.7,0.200
1,2019-11-13,@,vanderbilt,W,64,51,27,65,0.415,6,23,0.261,4,7,0.571,11,32,17,16,5,15,9,87.3,69.6,73.0,0.108,0.354,0.468,48.5,63.0,21.8,10.4,0.462,18.0,31.4,0.062
2,2019-11-17,@,temple,W,83,54,31,63,0.492,8,22,0.364,13,16,0.813,11,34,21,7,6,11,8,119.9,78.0,69.2,0.254,0.349,0.588,56.7,67.7,10.1,17.1,0.556,13.5,36.7,0.206
3,2019-11-19,H,virginia,W,83,44,30,59,0.508,9,23,0.391,14,22,0.636,9,37,21,11,7,10,13,117.6,62.4,70.6,0.373,0.390,0.598,54.4,70.0,15.6,17.1,0.585,12.6,31.0,0.237
4,2019-11-24,@,ohio-state,W,73,62,29,61,0.475,8,21,0.381,7,8,0.875,8,39,19,7,4,14,9,100.0,84.9,73.0,0.131,0.344,0.563,54.2,65.5,9.6,10.5,0.541,17.8,26.7,0.115


## Generating Moving Averages
Next we will generate a csv adding the moving averages for each statistic

In [16]:
LATEST = 'LATEST'
META_LABELS = ['Date', 'Location', 'Opp key', 'W/L', 'Tm', 'Opp']
STAT_LABELS = ['FG', 'FGA', 'FG%','3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA']

In [17]:
def generate_moving_averages_for_school(school_key, season, keep_latest = False, span = 5, isWomens = False):
    file_path = get_team_season_file_path(school_key, season, f'{school_key}_merged.csv', isWomens)
    team_df = pd.read_csv(file_path)

    # Drop any rows with NULL values
    team_df.dropna(inplace=True)

    if keep_latest:
        # This logic makes it so we keep the latest statistics for an upcoming game
        copyLast = pd.DataFrame(team_df.tail(1).values, columns=team_df.columns)
        team_df = pd.concat([team_df, copyLast], ignore_index=True)
        team_df.loc[team_df.index[-1], 'Date'] = LATEST
        team_df.loc[team_df.index[-1], 'Opp key'] = LATEST


    for column in team_df.columns:
        if column in META_LABELS:
            continue

        # Simple moving averages
        team_df[f"{column}_SMA"] = team_df.loc[:, column].rolling(window=span).mean()
        team_df[f"{column}_SMA"] = team_df[f"{column}_SMA"].shift(1)

        # Cumulative moving average
        team_df[f"{column}_CMA"] = team_df.loc[:, column].expanding(min_periods=span).mean()
        team_df[f"{column}_CMA"] = team_df[f"{column}_CMA"].shift(1)

        # Exponential moving average
        team_df[f"{column}_EMA"] = team_df.loc[:, column].ewm(span=span, adjust=False).mean()
        team_df[f"{column}_EMA"] = team_df[f"{column}_EMA"].shift(1)

    # Drop any rows with NULL values (rows with no MA)
    team_df.dropna(inplace=True)
    
    ma_file_path = get_team_season_file_path(school_key, season, f'{school_key}_{span}ma.csv', isWomens)
    team_df.to_csv(ma_file_path, index=False)

def generate_moving_averages_for_single_season(season, keep_latest = False, span = 5, isWomens = False):
    teams = WOMENS_SR_SCHOOL_KEYS if isWomens else MENS_SR_SCHOOL_KEYS
    for school_key in tqdm(teams, unit=f'school ({season} ma csv)'):
        try:
            generate_moving_averages_for_school(school_key, season, keep_latest, span, isWomens)
        except FileNotFoundError:
            continue

def generate_moving_averages(seasons, keep_latest = False, span = 5, isWomens = False):
    for season in seasons:
        generate_moving_averages_for_single_season(season, keep_latest, span, isWomens)

In [18]:
# generate_moving_averages(['2020', '2021', '2022', '2023'], isWomens=True)

Quick visual verification:

In [19]:
school_key, season, span = 'connecticut', 2020, 5
file_path = get_team_season_file_path(school_key, season, f'{school_key}_{span}ma.csv', isWomens=True)
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,FG_SMA,FG_CMA,FG_EMA,FGA_SMA,FGA_CMA,FGA_EMA,FG%_SMA,FG%_CMA,FG%_EMA,3P_SMA,3P_CMA,3P_EMA,3PA_SMA,3PA_CMA,3PA_EMA,3P%_SMA,3P%_CMA,3P%_EMA,FT_SMA,FT_CMA,FT_EMA,FTA_SMA,FTA_CMA,FTA_EMA,FT%_SMA,FT%_CMA,FT%_EMA,ORB_SMA,ORB_CMA,ORB_EMA,TRB_SMA,TRB_CMA,TRB_EMA,AST_SMA,AST_CMA,AST_EMA,STL_SMA,STL_CMA,STL_EMA,BLK_SMA,BLK_CMA,BLK_EMA,TOV_SMA,TOV_CMA,TOV_EMA,PF_SMA,PF_CMA,PF_EMA,ORtg_SMA,ORtg_CMA,ORtg_EMA,DRtg_SMA,DRtg_CMA,DRtg_EMA,Pace_SMA,Pace_CMA,Pace_EMA,FTr_SMA,FTr_CMA,FTr_EMA,3PAr_SMA,3PAr_CMA,3PAr_EMA,TS%_SMA,TS%_CMA,TS%_EMA,TRB%_SMA,TRB%_CMA,TRB%_EMA,AST%_SMA,AST%_CMA,AST%_EMA,STL%_SMA,STL%_CMA,STL%_EMA,BLK%_SMA,BLK%_CMA,BLK%_EMA,eFG%_SMA,eFG%_CMA,eFG%_EMA,TOV%_SMA,TOV%_CMA,TOV%_EMA,ORB%_SMA,ORB%_CMA,ORB%_EMA,FT/FGA_SMA,FT/FGA_CMA,FT/FGA_EMA
0,2019-11-26,@,dayton,W,75,37,31,62,0.500,10,19,0.526,3,4,0.750,8,44,19,7,8,12,8,107.3,53.0,69.9,0.065,0.306,0.587,57.9,61.3,10.0,18.6,0.581,15.8,26.7,0.048,28.8,28.800000,28.925926,62.6,62.600000,62.037037,0.4610,0.461000,0.467074,7.2,7.200000,7.432099,20.8,20.800000,20.604938,0.3460,0.346000,0.359370,10.2,10.200000,10.333333,14.8,14.800000,14.765432,0.7028,0.702800,0.732111,9.6,9.600000,9.160494,36.0,36.000000,36.925926,18.0,18.000000,18.160494,10.2,10.200000,9.370370,5.6,5.600000,5.456790,12.0,12.000000,11.975309,10.2,10.200000,10.333333,104.30,104.300000,104.953086,75.36,75.360000,76.774074,72.04,72.040000,72.180247,0.2378,0.237800,0.238654,0.3336,0.333600,0.33363,0.5394,0.539400,0.548704,52.90,52.900000,53.360494,62.12,62.120000,62.411111,14.10,14.100000,12.962963,13.68,13.680000,13.487654,0.5196,0.519600,0.528012,14.74,14.740000,14.841975,30.30,30.300000,29.403704,0.1640,0.164000,0.167148
1,2019-12-05,@,seton-hall,W,92,78,34,66,0.515,8,20,0.400,16,19,0.842,14,40,18,3,4,14,10,126.1,106.9,73.0,0.288,0.303,0.613,66.7,52.9,4.1,12.1,0.576,15.7,50.0,0.242,29.6,29.166667,29.617284,62.0,62.500000,62.024691,0.4780,0.467500,0.478049,8.2,7.666667,8.288066,21.6,20.500000,20.069959,0.3846,0.376000,0.414914,8.2,9.000000,7.888889,11.4,13.000000,11.176955,0.7290,0.710667,0.738074,9.4,9.333333,8.773663,37.2,37.333333,39.283951,19.4,18.166667,18.440329,9.6,9.666667,8.580247,6.0,6.000000,6.304527,12.4,12.000000,11.983539,9.4,9.833333,9.555556,106.42,104.800000,105.735391,69.58,71.633333,68.849383,71.14,71.683333,71.420165,0.1862,0.209000,0.180770,0.3486,0.329000,0.32442,0.5608,0.547333,0.561469,54.34,53.733333,54.873663,65.50,61.983333,62.040741,13.42,13.416667,11.975309,14.74,14.500000,15.191770,0.5450,0.529833,0.545675,15.54,14.916667,15.161317,30.50,29.700000,28.502469,0.1336,0.144667,0.127432
2,2019-12-08,H,notre-dame,W,81,57,30,66,0.455,8,23,0.348,13,19,0.684,13,45,18,5,6,15,9,106.4,74.9,76.5,0.288,0.348,0.540,60.8,60.0,6.6,15.0,0.515,16.7,38.2,0.197,31.0,29.857143,31.078189,62.2,63.000000,63.349794,0.4980,0.474286,0.490366,8.6,7.714286,8.192044,21.0,20.428571,20.046639,0.4124,0.379429,0.409942,10.6,10.000000,10.592593,13.8,13.857143,13.784636,0.7832,0.729429,0.772716,10.0,10.000000,10.515775,38.8,37.714286,39.522634,19.6,18.142857,18.293553,7.0,8.714286,6.720165,5.8,5.714286,5.536351,12.2,12.285714,12.655693,9.6,9.857143,9.703704,114.18,107.842857,112.523594,77.04,76.671429,81.532922,71.14,71.871429,71.946776,0.2222,0.220286,0.216513,0.3384,0.325286,0.31728,0.5898,0.556714,0.578646,57.98,55.585714,58.815775,63.48,60.685714,58.993827,9.88,12.085714,9.350206,15.08,14.157143,14.161180,0.5678,0.536429,0.555783,15.08,15.028571,15.340878,34.22,32.600000,35.668313,0.1696,0.158571,0.165621
3,2019-12-16,@,depaul,W,84,74,35,64,0.547,8,20,0.400,6,8,0.750,14,46,21,5,4,20,12,114.2,100.6,73.9,0.125,0.313,0.619,64.8,60.0,6.8,9.3,0.609,22.8,50.0,0.094,30.8,29.875000,30.718793,62.8,63.375000,64.233196,0.4906,0.471875,0.478578,8.6,7.750000,8.128029,21.2,20.750000,21.031093,0.4092,0.375500,0.389295,10.6,10.375000,11.395062,14.4,14.500000,15.523091,0.7574,0

## Merge opponent data

In [20]:
all_stat_cols = [item for col in STAT_LABELS for item in [col, f'{col}_SMA', f'{col}_CMA', f'{col}_EMA']]
opposing_stat_cols = [item for col in STAT_LABELS for item in [f'opp_{col}', f'opp_{col}_SMA', f'opp_{col}_CMA', f'opp_{col}_EMA']]
rename_opposing_cols = {item: f'opp_{item}' for stat in STAT_LABELS for item in [f'{stat}', f'{stat}_SMA', f'{stat}_CMA', f'{stat}_EMA']}

def merge_opponent_data_for_school(school_key, season, span = 5, isWomens = False):
    file_path = get_team_season_file_path(school_key, season, f'{school_key}_{span}ma.csv', isWomens)
    team_df = pd.read_csv(file_path)

    if team_df.shape[0] < 1:
        return

    home_df, away_df = pd.DataFrame(), pd.DataFrame()
    for index, row in team_df.iterrows():
        try:
            game_obj = row.to_dict()
            opponent_key = game_obj.get('Opp key')

            opponent_file_path = get_team_season_file_path(opponent_key, season, f'{opponent_key}_{span}ma.csv', isWomens)
            opponent_df = pd.read_csv(opponent_file_path)

            opponent_df = opponent_df.loc[(opponent_df['Opp key'] == school_key) & (opponent_df['Date'] == game_obj.get('Date'))]
            current_df = team_df[(team_df['Opp key'] == game_obj.get('Opp key')) & (team_df['Date'] == game_obj.get('Date'))]

            if game_obj.get('Location') == '@':
                home_df, away_df = pd.concat([home_df, opponent_df]), pd.concat([away_df, current_df])
            else:
                home_df, away_df = pd.concat([home_df, current_df]), pd.concat([away_df, opponent_df])
        except FileNotFoundError:
            continue

    # flip score column names for away dataframe to match home dataframe
    away_df.rename(columns={'Tm': 'Opp', 'Opp': 'Tm'}, inplace=True)

    away_df = away_df.drop(['Location', 'Opp key', 'W/L'], axis=1)
    away_df.rename(columns=rename_opposing_cols, inplace=True)

    merged_df = pd.merge(home_df, away_df, on=["Date", "Tm", "Opp"])
    merged_df = merged_df.sort_values(by='Date')

    merged_file_path = get_team_season_file_path(school_key, season, f'{school_key}_{span}span_full.csv', isWomens)
    merged_df.to_csv(merged_file_path, index=False)

def merge_opponent_data_for_single_season(season, span = 5, isWomens = False):
    teams = WOMENS_SR_SCHOOL_KEYS if isWomens else MENS_SR_SCHOOL_KEYS
    for school_key in tqdm(teams, unit=f'school ({season} full csv)'):
        try:
            merge_opponent_data_for_school(school_key, season, span, isWomens)
        except FileNotFoundError:
            continue

def merge_opponent_data(seasons, span = 5, isWomens = False):
    for season in seasons:
        merge_opponent_data_for_single_season(season, span, isWomens)

In [21]:
# merge_opponent_data(['2020', '2021', '2022', '2023'], isWomens=True)

Quick visual verification:

In [22]:
school_key, season, span = 'connecticut', 2023, 5
file_path = get_team_season_file_path(school_key, season, f'{school_key}_{span}span_full.csv', isWomens=True)
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,FG_SMA,FG_CMA,FG_EMA,FGA_SMA,FGA_CMA,FGA_EMA,FG%_SMA,FG%_CMA,FG%_EMA,3P_SMA,3P_CMA,3P_EMA,3PA_SMA,3PA_CMA,3PA_EMA,3P%_SMA,3P%_CMA,3P%_EMA,FT_SMA,FT_CMA,FT_EMA,FTA_SMA,FTA_CMA,FTA_EMA,FT%_SMA,FT%_CMA,FT%_EMA,ORB_SMA,ORB_CMA,ORB_EMA,TRB_SMA,TRB_CMA,TRB_EMA,AST_SMA,AST_CMA,AST_EMA,STL_SMA,STL_CMA,STL_EMA,BLK_SMA,BLK_CMA,BLK_EMA,TOV_SMA,TOV_CMA,TOV_EMA,PF_SMA,PF_CMA,PF_EMA,ORtg_SMA,ORtg_CMA,ORtg_EMA,DRtg_SMA,DRtg_CMA,DRtg_EMA,Pace_SMA,Pace_CMA,Pace_EMA,FTr_SMA,FTr_CMA,FTr_EMA,3PAr_SMA,3PAr_CMA,3PAr_EMA,TS%_SMA,TS%_CMA,TS%_EMA,TRB%_SMA,TRB%_CMA,TRB%_EMA,AST%_SMA,AST%_CMA,AST%_EMA,STL%_SMA,STL%_CMA,STL%_EMA,BLK%_SMA,BLK%_CMA,BLK%_EMA,eFG%_SMA,eFG%_CMA,eFG%_EMA,TOV%_SMA,TOV%_CMA,TOV%_EMA,ORB%_SMA,ORB%_CMA,ORB%_EMA,FT/FGA_SMA,FT/FGA_CMA,FT/FGA_EMA,opp_FG,opp_FGA,opp_FG%,opp_3P,opp_3PA,opp_3P%,opp_FT,opp_FTA,opp_FT%,opp_ORB,opp_TRB,opp_AST,opp_STL,opp_BLK,opp_TOV,opp_PF,opp_ORtg,opp_DRtg,opp_Pace,opp_FTr,opp_3PAr,opp_TS%,opp_TRB%,opp_AST%,opp_STL%,opp_BLK%,opp_eFG%,opp_TOV%,opp_ORB%,opp_FT/FGA,opp_FG_SMA,opp_FG_CMA,opp_FG_EMA,opp_FGA_SMA,opp_FGA_CMA,opp_FGA_EMA,opp_FG%_SMA,opp_FG%_CMA,opp_FG%_EMA,opp_3P_SMA,opp_3P_CMA,opp_3P_EMA,opp_3PA_SMA,opp_3PA_CMA,opp_3PA_EMA,opp_3P%_SMA,opp_3P%_CMA,opp_3P%_EMA,opp_FT_SMA,opp_FT_CMA,opp_FT_EMA,opp_FTA_SMA,opp_FTA_CMA,opp_FTA_EMA,opp_FT%_SMA,opp_FT%_CMA,opp_FT%_EMA,opp_ORB_SMA,opp_ORB_CMA,opp_ORB_EMA,opp_TRB_SMA,opp_TRB_CMA,opp_TRB_EMA,opp_AST_SMA,opp_AST_CMA,opp_AST_EMA,opp_STL_SMA,opp_STL_CMA,opp_STL_EMA,opp_BLK_SMA,opp_BLK_CMA,opp_BLK_EMA,opp_TOV_SMA,opp_TOV_CMA,opp_TOV_EMA,opp_PF_SMA,opp_PF_CMA,opp_PF_EMA,opp_ORtg_SMA,opp_ORtg_CMA,opp_ORtg_EMA,opp_DRtg_SMA,opp_DRtg_CMA,opp_DRtg_EMA,opp_Pace_SMA,opp_Pace_CMA,opp_Pace_EMA,opp_FTr_SMA,opp_FTr_CMA,opp_FTr_EMA,opp_3PAr_SMA,opp_3PAr_CMA,opp_3PAr_EMA,opp_TS%_SMA,opp_TS%_CMA,opp_TS%_EMA,opp_TRB%_SMA,opp_TRB%_CMA,opp_TRB%_EMA,opp_AST%_SMA,opp_AST%_CMA,opp_AST%_EMA,opp_STL%_SMA,opp_STL%_CMA,opp_STL%_EMA,opp_BLK%_SMA,opp_BLK%_CMA,opp_BLK%_EMA,opp_eFG%_SMA,opp_eFG%_CMA,opp_eFG%_EMA,opp_TOV%_SMA,opp_TOV%_CMA,opp_TOV%_EMA,opp_ORB%_SMA,opp_ORB%_CMA,opp_ORB%_EMA,opp_FT/FGA_SMA,opp_FT/FGA_CMA,opp_FT/FGA_EMA
0,2022-12-02,H,providence,W,98,53,36,59,0.610,13,23,0.565,13,18,0.722,8,41,28,6,6,12,12,136.2,73.7,71.9,0.305,0.390,0.725,60.3,77.8,8.3,12.5,0.720,15.1,33.3,0.220,33.4,33.400000,33.901235,63.6,63.600000,64.259259,0.5250,0.525000,0.526901,8.0,8.000000,7.851852,20.2,20.200000,20.074074,0.4146,0.414600,0.405531,12.4,12.400000,11.382716,16.8,16.800000,15.432099,0.6816,0.681600,0.642901,10.2,10.200000,10.160494,34.6,34.600000,35.827160,21.4,21.400000,22.666667,10.6,10.600000,10.160494,3.4,3.400000,3.839506,14.8,14.800000,14.456790,15.8,15.800000,14.777778,114.02,114.020000,114.023457,82.12,82.120000,82.585185,76.38,76.380000,76.239506,0.2702,0.270200,0.247864,0.3132,0.313200,0.308815,0.6086,0.608600,0.607358,57.58,57.580000,58.658025,63.56,63.560000,66.104938,13.70,13.700000,13.159259,10.56,10.560000,12.070370,0.5880,0.588000,0.588086,17.12,17.120000,16.779012,36.74,36.740000,36.479012,0.1988,0.198800,0.183222,23,68,0.338,4,20,0.200,3,7,0.429,11,27,11,5,2,12,15,73.7,136.2,71.9,0.103,0.294,0.372,39.7,47.8,7.0,5.6,0.368,14.4,25.0,0.044,23.8,24.125000,25.893004,58.0,57.875000,60.180155,0.4080,0.419000,0.429846,5.2,4.875000,6.015089,17.4,15.375000,17.241427,0.3046,0.338875,0.362046,12.0,12.000000,11.847279,17.0,17.625000,16.994513,0.6996,0.670750,0.690275,8.6,8.875000,9.163237,37.8,36.750000,39.286694,12.6,12.000000,13.951075,6.2,6.875000,5.940558,3.4,3.375000,3.555556,15.6,14.250000,13.163237,11.4,11.250000,9.540009,89.84,91.662500,97.694879,73.66,76.012500,76.609145,72.28,71.212500,71.433882,0.3010,0.309500,0.285685,0.3000,0.265250,0.287582,0.4888,0.493625,0.509711,53.66,52.750000,54.792730,51.72,48.500000,52.770370,8.62,9.687500,8.326840,9.34,10.100000,9.777641,0.4522,0.461500,0.479488,19.16,17.762500,1

## Generate dataset

In [23]:
FINAL_FEATURES = [
    'FG_SMA', 'FG_CMA', 'FG_EMA', 'FGA_SMA', 'FGA_CMA', 'FGA_EMA', 'FG%_SMA', 'FG%_CMA', 'FG%_EMA', '3P_SMA', '3P_CMA', '3P_EMA', '3PA_SMA', '3PA_CMA', '3PA_EMA', '3P%_SMA', '3P%_CMA', '3P%_EMA', 'FT_SMA', 'FT_CMA', 'FT_EMA', 
    'FTA_SMA', 'FTA_CMA', 'FTA_EMA', 'FT%_SMA', 'FT%_CMA', 'FT%_EMA', 'ORB_SMA', 'ORB_CMA', 'ORB_EMA', 'TRB_SMA', 'TRB_CMA', 'TRB_EMA', 'AST_SMA', 'AST_CMA', 'AST_EMA', 'STL_SMA', 'STL_CMA', 'STL_EMA', 'BLK_SMA', 'BLK_CMA', 
    'BLK_EMA', 'TOV_SMA', 'TOV_CMA', 'TOV_EMA', 'PF_SMA', 'PF_CMA', 'PF_EMA', 'ORtg_SMA', 'ORtg_CMA', 'ORtg_EMA', 'DRtg_SMA', 'DRtg_CMA', 'DRtg_EMA', 'Pace_SMA', 'Pace_CMA', 'Pace_EMA', 'FTr_SMA', 'FTr_CMA', 'FTr_EMA', 
    '3PAr_SMA', '3PAr_CMA', '3PAr_EMA', 'TS%_SMA', 'TS%_CMA', 'TS%_EMA', 'TRB%_SMA', 'TRB%_CMA', 'TRB%_EMA', 'AST%_SMA', 'AST%_CMA', 'AST%_EMA', 'STL%_SMA', 'STL%_CMA', 'STL%_EMA', 'BLK%_SMA', 'BLK%_CMA', 'BLK%_EMA', 'eFG%_SMA', 
    'eFG%_CMA', 'eFG%_EMA', 'TOV%_SMA', 'TOV%_CMA', 'TOV%_EMA', 'ORB%_SMA', 'ORB%_CMA', 'ORB%_EMA', 'FT/FGA_SMA', 'FT/FGA_CMA', 'FT/FGA_EMA', 'opp_FG_SMA', 'opp_FG_CMA', 'opp_FG_EMA', 'opp_FGA_SMA', 'opp_FGA_CMA', 'opp_FGA_EMA', 
    'opp_FG%_SMA', 'opp_FG%_CMA', 'opp_FG%_EMA', 'opp_3P_SMA', 'opp_3P_CMA', 'opp_3P_EMA', 'opp_3PA_SMA', 'opp_3PA_CMA', 'opp_3PA_EMA', 'opp_3P%_SMA', 'opp_3P%_CMA', 'opp_3P%_EMA', 'opp_FT_SMA', 'opp_FT_CMA', 'opp_FT_EMA', 
    'opp_FTA_SMA', 'opp_FTA_CMA', 'opp_FTA_EMA', 'opp_FT%_SMA', 'opp_FT%_CMA', 'opp_FT%_EMA', 'opp_ORB_SMA', 'opp_ORB_CMA', 'opp_ORB_EMA', 'opp_TRB_SMA', 'opp_TRB_CMA', 'opp_TRB_EMA', 'opp_AST_SMA', 'opp_AST_CMA', 'opp_AST_EMA', 
    'opp_STL_SMA', 'opp_STL_CMA', 'opp_STL_EMA', 'opp_BLK_SMA', 'opp_BLK_CMA', 'opp_BLK_EMA', 'opp_TOV_SMA', 'opp_TOV_CMA', 'opp_TOV_EMA', 'opp_PF_SMA', 'opp_PF_CMA', 'opp_PF_EMA', 'opp_ORtg_SMA', 'opp_ORtg_CMA', 'opp_ORtg_EMA', 
    'opp_DRtg_SMA', 'opp_DRtg_CMA', 'opp_DRtg_EMA', 'opp_Pace_SMA', 'opp_Pace_CMA', 'opp_Pace_EMA', 'opp_FTr_SMA', 'opp_FTr_CMA', 'opp_FTr_EMA', 'opp_3PAr_SMA', 'opp_3PAr_CMA', 'opp_3PAr_EMA', 'opp_TS%_SMA', 'opp_TS%_CMA', 
    'opp_TS%_EMA', 'opp_TRB%_SMA', 'opp_TRB%_CMA', 'opp_TRB%_EMA', 'opp_AST%_SMA', 'opp_AST%_CMA', 'opp_AST%_EMA', 'opp_STL%_SMA', 'opp_STL%_CMA', 'opp_STL%_EMA', 'opp_BLK%_SMA', 'opp_BLK%_CMA', 'opp_BLK%_EMA', 'opp_eFG%_SMA', 
    'opp_eFG%_CMA', 'opp_eFG%_EMA', 'opp_TOV%_SMA', 'opp_TOV%_CMA', 'opp_TOV%_EMA', 'opp_ORB%_SMA', 'opp_ORB%_CMA', 'opp_ORB%_EMA', 'opp_FT/FGA_SMA', 'opp_FT/FGA_CMA', 'opp_FT/FGA_EMA', 'Neutral', 'Win'
]

def generate_season_dataset(season, span = 5, isWomens = False):
    all_data_df = pd.DataFrame()

    teams = WOMENS_SR_SCHOOL_KEYS if isWomens else MENS_SR_SCHOOL_KEYS
    for school_key in tqdm(teams):
        file_path = get_team_season_file_path(school_key, season, f'{school_key}_{span}span_full.csv', isWomens)

        if os.path.exists(file_path):
            team_df = pd.read_csv(file_path)
            all_data_df = pd.concat([all_data_df, team_df])
        else:
            continue

    # sort by date
    all_data_df = all_data_df.sort_values(by="Date")

    # Add feature for if game is neutral site
    all_data_df['Neutral'] = all_data_df['Location'].apply(lambda x: 1 if x == 'N' else 0)

    # add label for win (1 = win, 0 = loss)
    all_data_df['Win'] = (all_data_df['Tm'] > all_data_df['Opp']).astype(int)

    # remove meta columns
    all_data_df = all_data_df.drop(META_LABELS, axis=1)

    # remove non moving average columns
    all_data_df = all_data_df.drop(STAT_LABELS, axis=1)
    all_data_df = all_data_df.drop([f'opp_{col}' for col in STAT_LABELS], axis=1)

    # Drop any rows with NULL value
    all_data_df.dropna(inplace=True)

    # Drop any duplicate rows
    all_data_df.drop_duplicates(inplace=True)

    # Reorder columns
    all_data_df = all_data_df.reindex(FINAL_FEATURES, axis=1)

    assert all([all_data_df.columns.to_list()[i] == FINAL_FEATURES[i] for i in range(len(FINAL_FEATURES))])

    type = 'women' if isWomens else 'men'
    training_data_path = os.path.abspath(f'../../data/dataset/{type}/{season}_{span}span_dataset.csv')
    all_data_df.to_csv(training_data_path, index=False)

def generate_datasets(seasons, span = 5, isWomens = False):
    for season in seasons:
        generate_season_dataset(season, span, isWomens)

In [24]:
# generate_datasets(['2020', '2021', '2022', '2023'], isWomens=True)

## Create full train test split
A typical train-test split for machine learning models is to use 70% of the data for training and 30% for testing.

In [25]:
def generate_test_train(split_neutral = False, span = 5, isWomens = False):
    type = 'women' if isWomens else 'men'
    data_dir_path = os.path.abspath(f'../../data/dataset/{type}/')
    pattern = r"\d{4}_" + str(span) + r"span_dataset\.csv"
    filenames = [filename for filename in os.listdir(data_dir_path) if re.match(pattern, filename)]
    merged_df = pd.concat([pd.read_csv(os.path.join(data_dir_path, filename)) for filename in filenames], ignore_index=True)

    print(f'Original shape: {merged_df.shape}')
    merged_df.dropna(inplace=True)
    print(f'Shape after dropping rows with null values: {merged_df.shape}')
    merged_df.drop_duplicates(inplace=True)
    print(f'Shape after dropping duplicate rows: {merged_df.shape}')

    if split_neutral:
        # Split the DataFrame into two based on 'Neutral' column
        neutral_df = merged_df[merged_df['Neutral'] == 1]
        home_away_df = merged_df[merged_df['Neutral'] == 0]

        neutral_train_df, neutral_test_df = train_test_split(neutral_df, test_size=0.3)
        home_away_train_df, home_away_test_df = train_test_split(home_away_df, test_size=0.3)

        print(len(neutral_train_df), 'neutral train examples')
        print(len(neutral_test_df), 'neutral test examples')
        print(len(home_away_train_df), 'home/away train examples')
        print(len(home_away_test_df), 'home/away test examples')

        neutral_train_df.to_csv(os.path.join(data_dir_path, f'{span}span_neutral_training_set.csv'), index=False)
        neutral_test_df.to_csv(os.path.join(data_dir_path, f'{span}span_neutral_testing_set.csv'), index=False)
        home_away_train_df.to_csv(os.path.join(data_dir_path, f'{span}span_home_away_training_set.csv'), index=False)
        home_away_test_df.to_csv(os.path.join(data_dir_path, f'{span}span_home_away_testing_set.csv'), index=False)
    else:
        train_df, test_df = train_test_split(merged_df, test_size=0.3)

        print(len(train_df), 'train examples')
        print(len(test_df), 'test examples')

        train_df.to_csv(os.path.join(data_dir_path, f'{span}span_training_set.csv'), index=False)
        test_df.to_csv(os.path.join(data_dir_path, f'{span}span_testing_set.csv'), index=False)

In [26]:
# generate_test_train(isWomens=True)

In [27]:
# isWomens = True
# spans = [3, 5, 7]
# for span in spans:
#     generate_moving_averages(['2020', '2021', '2022', '2023'], span=span, isWomens=isWomens)
#     merge_opponent_data(['2020', '2021', '2022', '2023'], span=span, isWomens=isWomens)
#     generate_datasets(['2020', '2021', '2022', '2023'], span=span, isWomens=isWomens)
#     generate_test_train(split_neutral = False, span=span, isWomens=isWomens)

# Latest statistic for each team

In [28]:
def generate_latest_stats_dict(season = 2024, spans = [3, 5, 7], isWomens = False):
    download_gamelogs_for_single_season(season, isWomens)
    create_basic_gamelogs_for_single_season(season, isWomens)
    create_advanced_gamelogs_for_single_season(season, isWomens)
    combine_basic_advanced_gamelogs_for_single_season(season, isWomens)

    latest_stats = {}
    for span in spans:
        latest_stats[span] = {}
        generate_moving_averages_for_single_season(season, keep_latest = True, span = span, isWomens = isWomens)
    
        teams = WOMENS_SR_SCHOOL_KEYS if isWomens else MENS_SR_SCHOOL_KEYS
        for school_key in tqdm(teams):
            file_path = get_team_season_file_path(school_key, season, f'{school_key}_{span}ma.csv', isWomens)

            if os.path.exists(file_path):
                team_df = pd.read_csv(file_path)

                # latest date
                date = team_df.loc[(team_df['Date'] != LATEST) & (team_df['Opp key'] != LATEST)]['Date'].iat[-1]

                # only get latest stats
                team_df = team_df.loc[(team_df['Date'] == LATEST) & (team_df['Opp key'] == LATEST)]

                assert team_df.shape[0] == 1

                # remove meta columns
                team_df = team_df.drop(META_LABELS, axis=1)

                # remove non moving average columns
                team_df = team_df.drop(STAT_LABELS, axis=1)

                # Reorder columns
                final_features_no_opp = [feature for feature in FINAL_FEATURES if 'opp_' not in feature and feature not in ['Neutral', 'Win']]
                team_df = team_df.reindex(final_features_no_opp, axis=1)

                assert len(team_df.columns.to_list()) == len(final_features_no_opp)
                assert all([team_df.columns.to_list()[i] == final_features_no_opp[i] for i in range(len(final_features_no_opp))])

                latest = team_df.iloc[-1:].to_numpy().tolist()
                latest_stats[span][school_key] = {'lastPlayed': date, 'stats': latest[0]}
    
    type = 'women' if isWomens else 'men'
    with open(f'{type}_data.json', 'w') as json_file:
        json.dump(latest_stats, json_file, indent=2)
        
    return latest_stats

In [29]:
# generate_latest_stats_dict(isWomens=False)
# generate_latest_stats_dict(isWomens=True)

100%|██████████| 370/370 [00:04<00:00, 90.50it/s]


{3: {'abilene-christian': {'lastPlayed': '2024-03-13',
   'stats': [21.33333333333333,
    24.633333333333333,
    20.41334862075746,
    58.333333333333336,
    59.96666666666667,
    57.76797498203814,
    0.363,
    0.4090666666666667,
    0.3494376937653869,
    8.0,
    8.6,
    7.834924343973398,
    24.666666666666668,
    28.23333333333333,
    23.88307026401162,
    0.3226666666666667,
    0.3013666666666666,
    0.3285663945134729,
    8.0,
    12.066666666666666,
    7.731143103912473,
    9.666666666666666,
    16.8,
    9.838901214301586,
    0.8393333333333333,
    0.7377666666666667,
    0.7967813140805811,
    9.333333333333334,
    11.133333333333333,
    8.165720354765654,
    31.0,
    33.96666666666667,
    29.705248961225152,
    16.0,
    14.233333333333333,
    14.795887807384132,
    8.666666666666666,
    7.666666666666667,
    8.478196704760194,
    1.3333333333333333,
    1.8,
    1.5043667033314705,
    14.0,
    15.133333333333333,
    12.699367068707945,
 

# Top 25

In [37]:
# mens_models = [model_filename.split('.pkl')[0] for model_filename in os.listdir('../../machine-learning/model/mens/')]
# womens_models = [model_filename.split('.pkl')[0] for model_filename in os.listdir('../../machine-learning/model/womens/')]
mens_models = ["3span_ensemble", "5span_ensemble", "7span_ensemble"]
womens_models = ["3span_ensemble", "5span_ensemble", "7span_ensemble"]

mens_ap_top_25_teams = [
    "connecticut", "houston", "purdue", "iowa-state", "north-carolina",
    "tennessee", "auburn", "marquette", "arizona", "illinois",
    "creighton", "kentucky", "duke", "baylor", "saint-marys-ca",
    "south-carolina", "kansas", "gonzaga", "alabama", "utah-state",
    "brigham-young", "texas-tech", "wisconsin", "san-diego-state", "washington-state"
]

womens_ap_top_25_teams = [
    "south-carolina", "iowa", "southern-california", "texas", "stanford",
    "ucla", "ohio-state", "louisiana-state", "notre-dame", "connecticut",
    "north-carolina-state", "oregon-state", "virginia-tech", "indiana", "kansas-state",
    "gonzaga", "colorado", "oklahoma", "baylor", "nevada-las-vegas",
    "utah", "syracuse", "louisville", "creighton", "fairfield"
]

def generate_top25(models, top_25_teams, isWomens = False):
    top_25_matchups = list(permutations(top_25_teams, 2))

    api_body = []
    for team1, team2 in top_25_matchups:
        for model in models:
            for isNeutral in [True, False]:
                api_body.append({"model": model, "isNeutral": isNeutral, "team1": team1, "team2": team2, "isWomens": isWomens})

    def chunk_list(lst, chunk_size):
        return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

    chunks = chunk_list(api_body, 60)

    all_predictions = []
    for chunk in tqdm(chunks):
        response = requests.post(url='http://mlmb-api.azurewebsites.net/predict', json=chunk)
        json_data = response.json()
        all_predictions += json_data

    scores = {}
    for prediction in all_predictions:
        team1_score = scores.get(prediction['team1'], 0)
        team2_score = scores.get(prediction['team2'], 0)
        scores[prediction['team1']] = round(team1_score + prediction['predictProba'][0], 2)
        scores[prediction['team2']] = round(team2_score + prediction['predictProba'][1], 2)

    type = 'women' if isWomens else 'men'
    with open(f'{type}_top_25.json', 'w') as json_file:
        json.dump(scores, json_file, indent=2)
    return scores

# generate_top25(mens_models, mens_ap_top_25_teams, isWomens = False)
# generate_top25(womens_models, womens_ap_top_25_teams, isWomens = True)

100%|██████████| 60/60 [04:49<00:00,  4.83s/it]


{'south-carolina': 214.59,
 'iowa': 176.72,
 'southern-california': 137.32,
 'texas': 187.46,
 'stanford': 163.89,
 'ucla': 162.76,
 'ohio-state': 126.3,
 'louisiana-state': 162.17,
 'notre-dame': 145.8,
 'connecticut': 190.46,
 'north-carolina-state': 121.62,
 'oregon-state': 122.35,
 'virginia-tech': 135.26,
 'indiana': 158.73,
 'kansas-state': 118.4,
 'gonzaga': 158.99,
 'colorado': 134.42,
 'oklahoma': 103.15,
 'baylor': 116.86,
 'nevada-las-vegas': 183.83,
 'utah': 137.76,
 'syracuse': 91.37,
 'louisville': 88.84,
 'creighton': 135.84,
 'fairfield': 125.11}